In [1]:
# Import modules 
from scripts.dataset import Dataset
from scripts.classification_model import Classification_model
from scripts.classification_model import Hyper_params,Lr,Lr_reduce_on_plateau,Early_stop,Learning_params
from scripts.model_evaluator import Model_evaluator
from scripts import classifier_utils
from time import gmtime, strftime
from random import uniform
import sys
import os
import tensorflow as tf
from keras import backend as K
import keras.backend.tensorflow_backend as KTF
import numpy as np


def get_session(gpu_fraction=0.8):
    'Allocate gpu_fraction its memory for computattion'

    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

KTF.set_session(get_session())
tf.GraphKeys.VARIABLES = tf.GraphKeys.GLOBAL_VARIABLES
cur_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())

# This code supports tensorflow backend only
assert K.image_dim_ordering() == 'tf'

Using TensorFlow backend.


In [2]:
data_dir = 'data'
train_file = 'train_dataset.txt'
test_dir = 'test'
test_file = 'test_dataset.txt'
CLASS_NAMES = ['1521','1703','1707','1729','1751','1755','1837','2037','2525','2526']
PRETRAINED_MODEL = 'VGG19' #TODO: check using Resnet 50

In [11]:
# # # May run this this one time only

## Copy images to data dir where each class images are in a specific folder 
classifier_utils.copy_images_to_class_dirs(train_file, data_dir, CLASS_NAMES)

# # Split data to train val
classifier_utils.resize_and_train_val_split(data_dir, CLASS_NAMES)

['1521', '1703', '1707', '1729', '1751', '1755', '1837', '2037', '2525', '2526']
resizing images
data/full/1521
pre-processing data/full/1521 dir
data/full/1703
pre-processing data/full/1703 dir
data/full/1707
pre-processing data/full/1707 dir
data/full/1729
pre-processing data/full/1729 dir
data/full/1751
pre-processing data/full/1751 dir
data/full/1755
pre-processing data/full/1755 dir
data/full/1837
pre-processing data/full/1837 dir
data/full/2037
pre-processing data/full/2037 dir
data/full/2525
pre-processing data/full/2525 dir
data/full/2526
pre-processing data/full/2526 dir
data/full/train/1521 created
data/full/train/1703 created
data/full/train/1707 created
data/full/train/1729 created
data/full/train/1751 created
data/full/train/1755 created
data/full/train/1837 created
data/full/train/2037 created
data/full/train/2525 created
data/full/train/2526 created
data/full/val/1521 created
data/full/val/1703 created
data/full/val/1707 created
data/full/val/1729 created
data/full/val/1

In [3]:
# Create experiment dirs (weights, summary)
[experiments_dir, model_dir, weights_dir, summary_dir] = classifier_utils.create_experiment_directories()

print 'Creating train dataset'
train_dataset = Dataset(data_dir,'train', PRETRAINED_MODEL,
                        class_names=CLASS_NAMES,
                        augment_factor=2.0, fliplr=True,
                        shift_range=0.2, zoom_range=0.2)
print 'Creating val dataset'
val_dataset = Dataset(data_dir,'val', PRETRAINED_MODEL,
                        class_names=CLASS_NAMES)

# Init params for first time - initial guess
#Hyperparams
hyper_params = Hyper_params(lr=Lr(init=0.0001,decay=0.0), optimization_method='Adam',
                           momentum=0.9, weight_decay=0.0002, dropout_rate=0.35,
                           freeze_layer_id=21, architecture_head_type='no_layers',
                           is_use_batch_normalization=True, nb_filters_conv_1_1_first=128,
                           nb_filters_conv_1_1_second=32, fc_layer_size_1=128, fc_layer_size_2=64)
#Learning params
learning_params = Learning_params(nb_train_samples=train_dataset.nb_samples,
                                 nb_validation_samples=val_dataset.nb_samples,
                                 batch_size=32, nb_max_epoch=150, 
                                 nb_samples_per_epoch=train_dataset.nb_samples ,
                                 nb_validation_samples_per_epoch=val_dataset.nb_samples,
                                 ckpt_period=1, early_stop=Early_stop(is_use=True, min_delta=0.005,patience=15),
                                 lr_reduce_on_plateau=Lr_reduce_on_plateau(is_use=True, factor=0.5, min_lr=1e-08, patience=3))
# Init classification model
clf_model = Classification_model(train_dataset.nb_classes,
                 weights_dir, summary_dir,
                 'VGG19',False)


Creating train dataset
data/train
10 Classes detected:
['1521', '1703', '1707', '1729', '1751', '1755', '1837', '2037', '2525', '2526']
Loading 8793 unique images from data/train/ directory
Found 8793 images belonging to 10 classes.
loaded and augmented all images
train shape: (17586, 224, 224, 3)
Creating val dataset
data/val
10 Classes detected:
['1521', '1703', '1707', '1729', '1751', '1755', '1837', '2037', '2525', '2526']
Loading 1506 unique images from data/val/ directory
Found 1506 images belonging to 10 classes.
loaded and augmented all images
train shape: (1506, 224, 224, 3)
loading vgg 19 network


In [22]:
# # # Hyper-Parameter search - don't run if not tuning, takes a lot of time !!

# TODO: add iteration over network, augmentation on luminance, iteration over different augmentation factors


# Iterate over hyperparameter space - currently only vgg19
# hyperparams to iterate: head_type,# nb_filters_conv_1_1_first, nb_filters_conv_1_1_second, fc_layer_size_1,
# fc_layer_size_2, freeze_layer, learning rate, weight decay,dropout_rate, batch size, 
# Init hyperparams and learning params with relevant values
# run model - low number of epochs (1)
# calc scores
# print changed value only, and scores


## Wasn't feasible on my machine to run over all hyper parameters, so by try and error I chose some of them and 
## and tuned the rest
# head_type_opts = ['no_layers', 'one_fc_layer']
# nb_filters_conv_1_1_first_opts = [16,32,64,128,256]
# nb_filters_conv_1_1_second_opts = [8,16,32,64,128]
# fc_layer_size_1_opts = [32,64,128,256,512,1024]
# fc_layer_size_2_opts = [16,32,64,128,256,512]
freeze_layer_id_opts = [20,21]#for resnet need to edit to last layers
batch_size_opts = [16,32]

max_clf_acc = 0.0
max_clf_hyper_params = Hyper_params()
max_clf_learning_params = Learning_params()

learning_params.nb_max_epoch = 1 # restrict num epochs per test
is_save_ckpts = False

num_options = 2*2*3*3*3
count = 0
for freeze_layer_id in freeze_layer_id_opts:
    hyper_params.freeze_layer_id = freeze_layer_id
    for batch_size in batch_size_opts:        
        for i in range(0,3):
            for j in range(0,3):
                for k in range(0,3):                     
                    learning_rate = 10**uniform(-6,-1)
                    hyper_params.lr.init = learning_rate
                    weight_decay = 10**uniform(-6,0)
                    hyper_params.weight_decay = weight_decay
                    dropout_rate = uniform(0,1)
                    hyper_params.dropout_rate = dropout_rate
                   
                    print '\n\n'    
                    learning_params.batch_size = batch_size
                    print 'freeze_layer_id: ' + str(freeze_layer_id)
                    print 'learning_rate: ' + str(learning_rate)
                    print 'weight_decay: ' + str(weight_decay)
                    print 'dropout_rate: ' + str(dropout_rate)
                    print 'batch_size: ' + str(batch_size) +'\n'
                    count = count + 1
                    print 'processing option number ' + str(count) + ' out of ' + str(num_options)                                                            
                    clf_model.build_model(input_shape=train_dataset.images[0,:,:,:].shape,
                                          hyper_params=hyper_params)      
                    clf_model.compile_model(learning_params, is_save_ckpts)
                    clf_model.train(train_data=train_dataset.images,train_labels=train_dataset.labels,
                            validation_data=val_dataset.images, validation_labels=val_dataset.labels)
                    clf_model_evaluator = Model_evaluator(clf_model, train_dataset.class_names)
                    clf_score = clf_model_evaluator.get_classifier_score(dataset=val_dataset)
                    if clf_score[1] > max_clf_acc:
                        print 'C + str(clf_score[1])
                        max_clf_acc = clf_score[1]
                        max_clf_hyper_params = hyper_params
                        max_clf_learning_params = learning_params
                    clf_model.delete_model()

In [1]:
# Keep max clf_score set and params
# Train model again - larger number of epochs

# Load best parameter set
max_clf_learning_params = learning_params
max_clf_hyper_params = hyper_params
max_clf_hyper_params.freeze_layer_id = 21
max_clf_hyper_params.lr.init = 0.00102382988483
max_clf_hyper_params.weight_decay = 0.000173410239311
max_clf_hyper_params.dropout_rate = 0.491165771543
max_clf_learning_params.batch_size = 32

# Train over many epochs with early stop callback
learning_params.nb_max_epoch = 100 # full train on best clf found
is_save_ckpts = True

is_use_classes_weights = False
classes_weights = None
if is_use_classes_weights:
    classes_weights = train_dataset.classes_weights

clf_model.build_model(input_shape=train_dataset.images[0,:,:,:].shape,
                      hyper_params=max_clf_hyper_params)      
clf_model.compile_model(max_clf_learning_params, is_save_ckpts)
clf_model.train(train_dataset.images,train_dataset.labels,
        val_dataset.images, val_dataset.labels, classes_weights)

In [2]:
# Evaluate model
clf_model_evaluator = Model_evaluator(clf_model, train_dataset.class_names)
clf_score = clf_model_evaluator.get_classifier_score(dataset=val_dataset)
clf_model_evaluator = Model_evaluator(clf_model, train_dataset.class_names)
clf_model_evaluator.evaluate(dataset=val_dataset)

In [7]:
## Save  best model weights to best model directory
## All check points are in experiment directory, possible to load intermediate checkpoints if overfit occured
if not os.path.exists('best_model'):
        os.makedirs('best_model')
clf_model.model.save_weights('best_model/best_weights.h5')

In [ ]:
# # # Production - Train over full training set

In [5]:
# Load data
data_dir = 'data'
data_dir = os.path.join(data_dir,'full')
is_production = True
is_save_ckpts = False

print 'creating production_train dataset'
prod_train_dataset = Dataset(data_dir,'train', PRETRAINED_MODEL,
                        class_names=CLASS_NAMES,
                        is_production = True,
                        augment_factor=2.0, fliplr=True,
                        shift_range=0.2, zoom_range=0.2)

creating production_train dataset
data/full
10 Classes detected:
['1521', '1703', '1707', '1729', '1751', '1755', '1837', '2037', '2525', '2526']
Loading 10299 unique images from data/full/ directory
Found 10299 images belonging to 10 classes.
loaded and augmented all images
train shape: (20598, 224, 224, 3)


In [3]:
# Create and train model, start training or load from scratch from last checkpoint

# Had an unknown bug while loading and saving models, can't tell why this happened.
# This is why I init a new model and loaded weights instead of loading using json files

best_hyper_params = Hyper_params(lr=Lr(init=0.00102382988483,decay=0.0), optimization_method='Adam',
                           momentum=0.9, weight_decay=0.000173410239311, dropout_rate=0.491165771543,
                           freeze_layer_id=21, architecture_head_type='no_layers',
                           is_use_batch_normalization=True, nb_filters_conv_1_1_first=128,
                           nb_filters_conv_1_1_second=32, fc_layer_size_1=128, fc_layer_size_2=64)
best_learning_params = Learning_params(nb_train_samples=prod_train_dataset.nb_samples,
                                 batch_size=32, nb_max_epoch=150, 
                                 nb_samples_per_epoch=prod_train_dataset.nb_samples ,
                                 ckpt_period=1, early_stop=Early_stop(is_use=True, min_delta=0.005,patience=15),
                                 lr_reduce_on_plateau=Lr_reduce_on_plateau(is_use=True, factor=0.5, min_lr=1e-08, patience=3))
best_weights_dir = 'best_model'
best_summary_dir = best_weights_dir
best_model = Classification_model(10, best_weights_dir, best_summary_dir, PRETRAINED_MODEL,False)
best_model.build_model(input_shape=prod_train_dataset.images[0,:,:,:].shape,
                       hyper_params=best_hyper_params)

best_model.model.load_weights(os.path.join(best_weights_dir,'best_model/best_weights.h5'))
best_model.compile_model(best_learning_params, is_save_ckpts, is_production)
best_model.train(prod_train_dataset.images, prod_train_dataset.labels, is_production=True)

In [ ]:
# Save production model 
best_model.model.save_weights('best_model/production_weights.h5')

In [16]:
## Generate predictions on test set


In [3]:
# Copy images to test dir 

test_images = classifier_utils.load_test_images(test_file, test_dir,PRETRAINED_MODEL)


Copying images to test dir
pre-processing test/images dir
Found 4674 images belonging to 1 classes.


In [4]:
# Predict images and save
predictions = best_model.model.predict(test_images)
out_classes = np.argmax(predictions, axis=1)
classifier_utils.generate_output_file(test_file, 'results_test.txt', out_classes, CLASS_NAMES)